In [229]:
import sys
import argparse
import warnings
from pathlib import Path
from scripts.processing import Processor
from scripts.folder_structure import FolderStructure
from scripts.analyze import load_pyseer_hits
from scripts.analyze import AnalyzeResults
from scripts.figures import get_k_loci, FIGURE4_PANELA, FIGURE4_PANELB


%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

# workflow IO
input_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/kleb-prophage-div/2025-02-12_KLEBDATA_LIGHT"
output_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/gwas/2025-03-26_KLEBDATA_LIGHT_GWAS"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# folder structure
folder_manager = FolderStructure(input_dir, output_dir)
structure = folder_manager.get_paths_dict()
params = folder_manager.get_params_dict()

# aggregate results
processor = Processor(structure, params)
processor.aggregate_pyseer_hits()
processor.aggregate_function_prediction()

# load
pyseer_hits = load_pyseer_hits(structure, params)

Created directory: /Users/januszkoszucki/tmp_blast_dir


In [ ]:
analyze = AnalyzeResults(structure, params)

analyze.per_locus_analysis(run=True)
# analyze.aggregate_data(run=False)
# analyze.combine_predictions_and_enzymes(run=False)

In [65]:
# FIGURE4_PANELA(structure, params, merge_expr=True, target_fig_height=1.4, target_fig_width=7.5, \
#                labels_fontsize=4, cells_fontsize=3.5, margin=0.3, header_space=0.1)

# FIGURE4_PANELB(structure, params, min_cov=0.5, min_pident=0, max_evalue=1e-3, use_coverage=True)

In [36]:
tmp = '/Users/januszkoszucki/MGG Dropbox/Projects/gwas/2025-03-26_KLEBDATA_LIGHT_GWAS/1_INTERMEDIATE/3_FUNCTIONS/clusters_functions_best_all.tsv'
df = pd.read_csv(tmp, sep='\t')

In [55]:
df.query('version== "PCI50C80" and PC == "PC0347"')

,PC,version,PC80,db,function,reported_topology_PC80,reported_topology_PC,prob,bits,target,pvalue,ident,qcov,tcov,qstart,qend,qlength,tstart,tend,tlength,evalue,name,color,annot,category,phrog/alan_profile
263677,PC0347,PCI50C80,PC01526,ECOD,5050.1.1.2 | X: NO_X_NAME | H: NO_H_NAME | T: ...,other,other,0.995,152.2,ECOD_001680299_e4zowA1,5.100000e-23,0.14,0.44,0.86,9.0,180.0,394.0,3.0,175.0,202.0,2.200000e-18,ECOD_001680299_e4zowA1 | 5050.1.1.2 | 4ZOW A:1...,NaN,NaN,NaN,0
263678,PC0347,PCI50C80,PC01526,PFAM,PF00083.24 | Sugar (and other) transporter,other,other,0.999,258.5,PF00083.24,1.200000e-35,0.13,0.90,0.93,34.0,389.0,394.0,20.0,421.0,434.0,2.100000e-31,PF00083.24 ; Sugar_tr ; Sugar (and other) tran...,NaN,NaN,NaN,0
263679,PC0347,PCI50C80,PC01526,PFAM,PF00854.21 | POT family,other,other,0.998,201.0,PF00854.21,1.200000e-27,0.14,0.72,0.99,79.0,360.0,394.0,1.0,418.0,424.0,2.200000e-23,PF00854.21 ; PTR2 ; POT family,NaN,NaN,NaN,0
263680,PC0347,PCI50C80,PC01526,PFAM,PF01306.19 | LacY proton/sugar symporter,other,other,0.999,248.1,PF01306.19,1.200000e-34,0.12,0.91,0.91,34.0,391.0,394.0,28.0,403.0,412.0,2.100000e-30,PF01306.19 ; LacY_symp ; LacY proton/sugar sym...,NaN,NaN,NaN,0
263681,PC0347,PCI50C80,PC01526,PFAM,PF01770.18 | Reduced folate carrier,other,other,0.999,252.9,PF01770.18,1.300000e-34,0.10,0.81,0.91,32.0,352.0,394.0,22.0,413.0,432.0,2.400000e-30,PF01770.18 ; Folate_carrier ; Reduced folate c...,NaN,NaN,NaN,0
263682,PC0347,PCI50C80,PC01526,PFAM,PF02487.17 | CLN3 protein,other,other,0.948,65.6,PF02487.17,5.300000e-08,0.14,0.78,0.91,50.0,355.0,394.0,30.0,402.0,412.0,9.600000e-04,PF02487.17 ; CLN3 ; CLN3 protein,NaN,NaN,NaN,0
263683,PC0347,PCI50C80,PC01526,PFAM,PF03092.16 | BT1 family,other,other,0.997,208.6,PF03092.16,2.300000e-26,0.09,0.84,0.88,33.0,361.0,394.0,9.0,518.0,580.0,4.200000e-22,PF03092.16 ; BT1 ; BT1 family,NaN,NaN,NaN,0
263684,PC0347,PCI50C80,PC01526,PFAM,PF03137.20 | Organic Anion Transporter Polypep...,other,other,0.999,249.4,PF03137.20,3.600000e-32,0.15,0.85,0.94,28.0,363.0,394.0,16.0,543.0,564.0,6.400000e-28,PF03137.20 ; OATP ; Organic Anion Transporter ...,NaN,NaN,NaN,0
263685,PC0347,PCI50C80,PC01526,PFAM,PF03209.15 | PUCC protein,other,other,0.999,255.2,PF03209.15,1.100000e-35,0.14,0.90,1.00,36.0,390.0,394.0,2.0,403.0,404.0,1.900000e-31,PF03209.15 ; PUCC ; PUCC protein,NaN,NaN,NaN,0
263686,PC0347,PCI50C80,PC01526,PFAM,PF03219.14 | TLC ATP/ADP transporter,other,other,0.997,199.5,PF03219.14,3.000000e-26,0.10,0.84,0.86,35.0,363.0,394.0,38.0,443.0,474.0,5.400000e-22,PF03219.14 ; TLC ; TLC ATP/ADP transporter,NaN,NaN,NaN,0
